In [1]:
!pip install cryptography

In [2]:
from cryptography.fernet import Fernet

key = Fernet.generate_key()
cipher = Fernet(key)
encrypted = cipher.encrypt(b"Hello, Pavan!")
print("Encrypted:", encrypted)

decrypted = cipher.decrypt(encrypted)
print("Decrypted:", decrypted)

Encrypted: b'gAAAAABn_5PZqBsN2lDIT-AMBPLmmZiLfRpg-vtWqmS5We8K7q6yFv-JfKflp0OoIh8z7n3y2xwc3RLvnvEomJZaQu3rtjxyMQ=='
Decrypted: b'Hello, Pavan!'


In [8]:
from cryptography.fernet import Fernet

In [9]:
key = Fernet.generate_key()
cipher = Fernet(key)
print("Key generated successfully!")

Key generated successfully!


In [12]:
# Password-protected key generation
from cryptography.fernet import Fernet
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.backends import default_backend
import base64, os

def generate_encrypted_key(password: str):
    key = Fernet.generate_key()
    
    salt = os.urandom(16)
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=100000,
        backend=default_backend()
    )
    password_key = base64.urlsafe_b64encode(kdf.derive(password.encode()))
    f = Fernet(password_key)
    encrypted_key = f.encrypt(key)
    
    with open("enc_key.key", "wb") as file:
        file.write(salt + encrypted_key)
    
    print("Encrypted Fernet key saved as 'enc_key.key'")

In [13]:
def load_key(password: str):
    with open("enc_key.key", "rb") as file:
        data = file.read()
        salt = data[:16]
        encrypted_key = data[16:]
        
        kdf = PBKDF2HMAC(
            algorithm=hashes.SHA256(),
            length=32,
            salt=salt,
            iterations=100000,
            backend=default_backend()
        )
        password_key = base64.urlsafe_b64encode(kdf.derive(password.encode()))
        f = Fernet(password_key)
        key = f.decrypt(encrypted_key)
        return key

In [14]:
def encrypt_file(file_path, key):
    f = Fernet(key)
    with open(file_path, 'rb') as file:
        data = file.read()
    encrypted_data = f.encrypt(data)
    with open(file_path + ".enc", 'wb') as file:
        file.write(encrypted_data)
    print("File encrypted successfully!")

def decrypt_file(file_path, key):
    f = Fernet(key)
    with open(file_path, 'rb') as file:
        encrypted_data = file.read()
    decrypted_data = f.decrypt(encrypted_data)
    original_file = file_path.replace(".enc", "")
    with open(original_file, 'wb') as file:
        file.write(decrypted_data)
    print("File decrypted successfully!")

In [16]:
#GENERATE KEY
# Only run once
# generate_encrypted_key("yourpassword123")

# LOAD KEY
key = load_key("yourpassword123")

# ENCRYPT OR DECRYPT
encrypt_file("sample.txt", key)
# decrypt_file("sample.txt.enc", key)

FileNotFoundError: [Errno 2] No such file or directory: 'enc_key.key'

In [18]:
generate_encrypted_key("yourpassword123")

Encrypted Fernet key saved as 'enc_key.key'


In [20]:
key=load_key("yourpassword123")
encrypt_file("simple.txt",key)
#decrypt_file("simpple.txt.enc",key)

FileNotFoundError: [Errno 2] No such file or directory: 'simple.txt'

In [21]:
with open("simple.txt","w") as f:
    f.write("This is Pavan's encrypted file. only authotized eyes allowed!")

In [22]:
generate_encrypted_key("yourpassword123")

Encrypted Fernet key saved as 'enc_key.key'


In [25]:
key=load_key("yourpassword123")
encrypt_file("simple.txt",key)

File encrypted successfully!


In [26]:
import os
print(os.listdir())

['.git', '.ipynb_checkpoints', 'enc_key.key', 'FileEncryptionDecryptionTool.ipynb', 'simple.txt', 'simple.txt.enc']


In [27]:
decrypt_file("simple.txt.enc",key)

File decrypted successfully!


In [1]:
import hashlib

# Function to generate file hash (SHA-256)
def generate_file_hash(file_path):
    sha256_hash = hashlib.sha256()
    with open(file_path, "rb") as f:
        # Read and update hash string value in blocks of 4K
        for byte_block in iter(lambda: f.read(4096), b""):
            sha256_hash.update(byte_block)
    return sha256_hash.hexdigest()

# Function to verify file hash
def verify_file_hash(original_file, decrypted_file):
    original_hash = generate_file_hash(original_file)
    decrypted_hash = generate_file_hash(decrypted_file)
    
    if original_hash == decrypted_hash:
        print("File integrity verified. No tampering detected.")
    else:
        print("Warning: File tampering detected!")

In [2]:
import os

# Function to delete decrypted file
def delete_file(file_path):
    try:
        os.remove(file_path)
        print(f"File {file_path} deleted successfully.")
    except FileNotFoundError:
        print(f"File {file_path} not found.")

In [3]:
import time

# Function to check if the key has expired
def check_key_expiry(expiry_time):
    current_time = time.time()
    if current_time > expiry_time:
        print("The decryption key has expired. You cannot decrypt the file.")
        return False
    else:
        print("Key is valid.")
        return True

# Set an expiry time (for example, 1 hour from now)
expiry_time = time.time() + 3600  # 1 hour in the future

In [4]:
def decrypt_file_with_security(file_path, key, expiry_time=None):
    # Check if the key has expired
    if expiry_time and not check_key_expiry(expiry_time):
        return
    
    # Decrypt the file
    f = Fernet(key)
    with open(file_path, 'rb') as file:
        encrypted_data = file.read()
    decrypted_data = f.decrypt(encrypted_data)
    
    original_file = file_path.replace(".enc", "")
    with open(original_file, 'wb') as file:
        file.write(decrypted_data)
    
    # Verify file integrity (hash check)
    verify_file_hash(file_path.replace(".enc", ""), original_file)
    
    # Auto-delete decrypted file after use
    delete_file(original_file)
    
    print("File decrypted and deleted successfully!")

In [6]:
import os
import hashlib
import time
from cryptography.fernet import Fernet
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC

# Function to generate a key
def generate_key(password: str):
    # Generate a salt
    salt = os.urandom(16)
    
    # Use PBKDF2 to derive a key from the password and salt
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=100000,
        backend=default_backend()
    )
    
    key = kdf.derive(password.encode())  # Generate key
    # Save the key along with the salt in a file
    with open("enc_key.key", "wb") as key_file:
        key_file.write(salt + key)  # Store salt and key

# Function to load the key
def load_key(password: str):
    # Read the key and salt from the file
    with open("enc_key.key", "rb") as key_file:
        data = key_file.read()
        salt = data[:16]  # The first 16 bytes are the salt
        key = data[16:]  # The rest is the key
    
    # Derive the key from the password and salt
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=100000,
        backend=default_backend()
    )
    
    kdf.verify(password.encode(), key)  # Verify the password is correct
    return key

# Function to encrypt a file
def encrypt_file(file_path, key):
    f = Fernet(key)  # Initialize Fernet with the key
    with open(file_path, 'rb') as file:
        data = file.read()
    
    encrypted_data = f.encrypt(data)
    
    with open(file_path + ".enc", 'wb') as enc_file:
        enc_file.write(encrypted_data)

# Function to decrypt a file with security features
def decrypt_file_with_security(file_path, key, expiry_time):
    if time.time() > expiry_time:
        print("Error: The decryption key has expired!")
        return

    # Read the encrypted data
    f = Fernet(key)
    try:
        with open(file_path, 'rb') as enc_file:
            encrypted_data = enc_file.read()
        
        # Decrypt the data
        decrypted_data = f.decrypt(encrypted_data)
        
        # Hash to verify file integrity
        original_hash = hashlib.sha256(decrypted_data).hexdigest()
        stored_hash = hashlib.sha256(encrypted_data).hexdigest()
        
        if original_hash != stored_hash:
            print("Error: File integrity check failed!")
            return
        
        # Write the decrypted data to a new file
        with open(file_path[:-4], 'wb') as dec_file:  # Remove ".enc" extension
            dec_file.write(decrypted_data)
        
        print("File decrypted successfully")
        
        # Delete the decrypted file for auto-deletion
        os.remove(file_path[:-4])
        print("Decrypted file deleted automatically after processing.")
    except Exception as e:
        print(f"Error during decryption: {e}")

In [7]:
# Encrypt the file
key = load_key("yourpassword123")  # Load the encryption key
encrypt_file("simple.txt", key)  # Make sure to replace 'simple.txt' with your actual file name

InvalidKey: Keys do not match.

In [8]:
generate_key("yourpassword123")

In [9]:
key = load_key("yourpassword123")

In [10]:
encrypt_file("simple.txt",key)

ValueError: Fernet key must be 32 url-safe base64-encoded bytes.

In [11]:
from cryptography.fernet import Fernet

def generate_key():
    key = Fernet.generate_key()
    with open("enc_key.key", "wb") as key_file:
        key_file.write(key)
    print("Key generated and saved as 'enc_key.key'.")

generate_key()

Key generated and saved as 'enc_key.key'.


In [12]:
def load_key():
    with open("enc_key.key", "rb") as key_file:
        return key_file.read()

key = load_key()

In [13]:
def encrypt_file(file_path, key):
    f = Fernet(key)
    with open(file_path, "rb") as file:
        data = file.read()
    encrypted_data = f.encrypt(data)
    with open(file_path + ".enc", "wb") as file:
        file.write(encrypted_data)
    print("File encrypted successfully.")

encrypt_file("simple.txt", key)

File encrypted successfully.


In [14]:
def decrypt_file(file_path, key):
    f = Fernet(key)
    with open(file_path, "rb") as file:
        encrypted_data = file.read()
    decrypted_data = f.decrypt(encrypted_data)

    # Save the decrypted file (you can choose a different name if needed)
    with open("decrypted_" + file_path.replace(".enc", ""), "wb") as file:
        file.write(decrypted_data)
    print("File decrypted successfully.")

# Call the function
decrypt_file("simple.txt.enc", key)

File decrypted successfully.
